In [1]:
# Snippets that may be useful for report writing

In [ ]:
def perturb_numerical(value, column_data, numeric_displacement=0.1):
    
    # We use this to ensure zero values are perturbed
    ADDITIONAL_VALUE = 0.0001
    
    # Decide whether to increase or decrease
    direction = random.randrange(2)
    
    # If increasing - round up
    if (direction == 0):
        new_value = value * (1 + numeric_displacement) + ADDITIONAL_VALUE
        new_value = math.ceil(new_value)
        
    # If decreasing - round down
    else:
        new_value = value * (1 - numeric_displacement) - ADDITIONAL_VALUE
        new_value = math.floor(new_value)
    
    # Check no higher/lower than max/min values
    if ('max' in column_data):
        new_value = min(new_value, column_data['max'])
    if ('min' in column_data):
        new_value = max(new_value, column_data['min'])

    return new_value


def perturb_ordinal(value, column_data):
    
    # Get total number of values
    num_values = len(column_data['values'])
    
    # Work out index of value
    value_index = 0
    for idx, val in enumerate(column_data['values']):
        if val == value:
            value_index = idx
            break
        
    # Decide whether to increase or decrease
    movement = 1 if random.random() < 0.5 else -1
    
    # Return value, looping to other end if required
    if value_index + movement < 0:
        return column_data['values'][num_values - 1]
    
    elif value_index + movement >= num_values:
        return column_data['values'][0]
    
    else:
        return column_data['values'][value_index + movement]
    
def perturb_nominal(value, column_data):
    
    # Remove actual value from the list of values
    other_values = list(filter((lambda v: v != value), column_data['values']))
    
    # Randomly select a new value from this new list, return
    return random.choice(other_values)

In [ ]:
# More sophisticated implementation of sensitivity

# TODO: could PSO be used for this to get to the global maximum? 

def calculate_sensitivity(metadata_local, df_local, numeric_displacement=0.1, proportion_features_perturbed=0.1, perturbation_epochs=10):
        
    # Filter our features not used
    used_features = list(filter((lambda feat: feat['type'] != 'outcome' and feat['used'] == True), metadata_local))
    
    # Keep track of the maximum explanation difference
    max_difference = 0
                
    # Calculate how many features to perturb
    n = math.ceil(len(used_features) * proportion_features_perturbed)
    
    # Loop through each perturbation epoch
    for p_epoch in range(perturbation_epochs):
                
        # Randomly decide which features to peturb
        features_to_perturb = random.sample(used_features, n)
        
        # Make a copy of the instance
        instance_copy = df_local.iloc[index_to_explain,:].copy(deep=True)
        
        # Apply the perturbations, varying the technique depending on the data type
        for column in features_to_perturb:
                        
            if column['type'] == 'numerical':
                instance_copy[column['index']] = perturb_numerical(instance_copy[column['index']], column)
            
            elif column['type'] == 'ordinal':
                instance_copy[column['index']] = perturb_ordinal(instance_copy[column['index']], column)
                
            elif column['type'] == 'nominal':
                instance_copy[column['index']] = perturb_nominal(instance_copy[column['index']], column)
            
            else:
                print('Error - not a recognised type')
        
        # Generate an explantion
        perturbed_shap_values = explainer.shap_values(instance_copy, nsamples=num_perturbations)
        
        # Calculate L2 distance between this and the original explanation
        difference = np.linalg.norm(np.array(perturbed_shap_values) - np.array(shap_values))

        if difference > max_difference:
            max_difference = difference

    return max_difference

results = []

for epochs in range(30):
    print(f'Starting with {epochs} epochs')
    sensitivity = calculate_sensitivity(metadata, dataframe, perturbation_epochs=epochs)
    results.append({
        'epochs': epochs,
        'sensitivity': sensitivity
    })


In [ ]:
# Other metrics for model training

#       tf.keras.metrics.Truenegatives(name='tp'),
#       tf.keras.metrics.Falsenegatives(name='fp'),
#       tf.keras.metrics.TrueNegatives(name='tn'),
#       tf.keras.metrics.FalseNegatives(name='fn'), 
#       tf.keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve


### Notes on the fidelity metric

It is a bit confusing. 

The first part is (perturbation) dot (explanation). The metric is made higher by these - in other words, an explanation that is more confident in a particular feature being important, and/or a large perturbation to that feature, should result in a larger change to the model output. This seems reasonable. 

The second part of the actual change in the model output, squared.

The end result is the first part minus the second part. 

A large negative value implies high FIdidelity (i.e. changing the inputs in line with the explanation has, indeed, resulted in a large change to model output) - this is presumably why Yeh. et al added the [in] to the name of the metric.

NB I'd previously written this TODO: *think about what happends if we are already at the mid-point for the variables we are perturbing? Do we need different approaches?*. We don't need different approaches, instead we rely on the first part of the formula, small perturbations = smaller expected change in the model output. 
